In [2]:
import pandas as pd
import numpy as np
import json

In [8]:
filePath = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/task43/csvData/asr2iot_iot2asr_robo_4.tsv"
dataDf = pd.read_csv(filePath,sep='\t')
# dataDf.columns = ["query", "domain", "intent", "slots"]
dataDf.head()

,query,domain,intent,slots
0,嗯帮我返回基站,narwal_other_function,start_back_station,[]
1,回去充电,narwal_other_function,start_back_station,[]
2,来我这里扫一下,narwal_clean_function,come_here_sweep,[]
3,结束任务结束任务,other,other,[]
4,继续工作,other,other,[]


In [5]:
dataDf.loc[0, "iot2asr"]

KeyError: 'iot2asr'

In [21]:
def slot_swich(slots, query):
    slotsList = []
    for slot_type in slots.keys():
        slottypeList = slots.get(slot_type, [])
        for slottype in slottypeList:
            text = slottype.get("text")
            # value = str(slottype.get("value"))
            if (slot_type == 'frequency'):
                startIndex = query.find(text)
                slotsList.append(['frequency', startIndex, startIndex+len(text)])
                continue
            if "和 " in text:
                # value = value.replace("he ", "")
                text = text.replace("和 ", "")
            text = text.split(" ")
            for i in range(len(text)):
                t = text[i]
                startIndex = query.find(t)
                if startIndex == -1:
                    print(f"query:{query}  slot:{text} ")
                    return None
                slotsList.append([slot_type, startIndex, startIndex+len(t)])
    return slotsList

dataList = []
def slots_deal(row):
    query = row["query"]
    x = row["asr2iot"]
    datas = json.loads(x)[0]
    domain, intent, slots = datas[0], datas[1], datas[2]
    slotsList = []
    if slots == {}: slotsList = None
    else: 
        slotsList = slot_swich(slots=slots, query=query)
        if slotsList is None:
            slotsList = "error"
    return [query, domain, intent, slotsList]
        
dataList = dataDf[["query", "asr2iot"]].apply(slots_deal, axis=1)

query:剪刀布  slot:['石头剪刀布'] 
query:重新开始  slot:['零秒'] 
query:广州天气  slot:['明天'] 
query:打扫沙发底下  slot:['沙发', 'under'] 
query:多少度  slot:['今天'] 
query:打扫沙发下面  slot:['沙发', 'under'] 
query:精细拖地  slot:['精细擦地'] 
query:打扫餐桌下面  slot:['餐桌', 'under'] 
query:扫一下餐桌底下  slot:['餐桌', 'under'] 
query:打扫床底下  slot:['床', 'under'] 
query:查看天气  slot:['明天'] 
query:沙发下面  slot:['沙发', 'under'] 
query:打扫附近  slot:['nearby'] 
query:打扫餐桌底下  slot:['餐桌', 'under'] 
query:几点下雨  slot:['今天'] 
query:打扫餐桌附近  slot:['餐桌', 'nearby'] 
query:what's your name  slot:["what'syourname"] 
query:打扫沙发附近  slot:['沙发', 'nearby'] 
query:打扫床底  slot:['床', 'under'] 
query:再来一遍  slot:['零秒'] 
query:清扫床底  slot:['床', 'under'] 
query:你天的天气  slot:['今天'] 
query:吸力调至MAX+  slot:['max加'] 
query:后面  slot:['behind'] 
query:清扫附近  slot:['nearby'] 
query:打扫周围  slot:['nearby'] 
query:下面  slot:['under'] 
query:打扫睡房  slot:['卧室'] 
query:扫下床底下  slot:['床', 'under'] 
query:穿衣指数  slot:['今天'] 
query:打扫沙发前面  slot:['沙发', 'front'] 
query:南通的天气预报  slot:['今天'] 
query:沙发周边边扫边拖

In [22]:
resultDf = pd.DataFrame(dataList.values.tolist(), columns=["query", "domain", "intent", "slots"])
resultDf.head()

,query,domain,intent,slots
0,返回基站,roborock_other_function,back_charge,None
1,开始清洁,roborock_clean_function,start_clean,None
2,暂停,sys_command,pause,None
3,开始清扫,roborock_clean_function,start_clean,None
4,回基站,roborock_other_function,back_charge,None


In [5]:
import yaml
import os
def parser_yml(ymlDir):
    ymlFilePaths = [ymlDir + "/" + ymlName for ymlName in os.listdir(ymlDir)]
    intentData = {}
    for ymlPath in ymlFilePaths:
        with open(ymlPath, 'r') as file:
            data = yaml.safe_load(file)
            topKey = list(data.keys())[0]
            intentData.setdefault(topKey, list(data.get(topKey).keys()))
    return intentData
dataInput = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/dags/temp/task43/inputData"
intentDict = parser_yml(dataInput)
intentList = []
for intentValues in intentDict.values(): 
    intentList += intentValues
intentList

FileNotFoundError: [Errno 2] No such file or directory: '/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/dags/temp/task43/inputData'

In [10]:
resultDf = resultDf[resultDf["slots"] != "error"]

NameError: name 'resultDf' is not defined

In [25]:
resultDf[resultDf["intent"].isin(intentList)].drop_duplicates(["query"], keep="first").to_csv("intent_static_data.tsv", index=False, sep="\t")

In [26]:
intentDataNum = []
for intent in intentList:
    dataNum = len(resultDf[resultDf["intent"] == intent])
    intentDataNum.append([intent, dataNum])
intentDataDf = pd.DataFrame(intentDataNum, columns=["intent", "count"])
intentDataDf.sort_values("count", ascending=False).to_csv("intent_data_num.tsv", index=False, sep="\t")


In [27]:
len(intentList)

109

### 云鲸处理

In [12]:
filePath = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/data/task43/csvData/query_from_asr2iot_narwal_6-11.tsv"
dataDf = pd.read_csv(filePath,sep='\t', header=None)
dataDf.columns = ["query", "domain", "intent", "slots"]
dataDf.head()

,query,domain,intent,slots
0,嗯帮我返回基站,narwal_other_function,start_back_station,[]
1,回去充电,narwal_other_function,start_back_station,[]
2,来我这里扫一下,narwal_clean_function,come_here_sweep,[]
3,结束任务结束任务,other,other,[]
4,继续工作,other,other,[]


In [13]:
dataInput = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/dags/temp/task43/inputData"
intentDict = parser_yml(dataInput)
intentList = []
for intentValues in intentDict.values(): 
    intentList += intentValues
intentList

['close_ai_recognition',
 'close_carpet_avoidance',
 'close_carpet_pressurization',
 'close_childlock',
 'close_dnd_mode',
 'close_laser_assistant',
 'close_station_light',
 'close_voice_assistant',
 'continue_back_station',
 'continue_clean_roller',
 'continue_collect_dust',
 'continue_drying',
 'continue_exit_duck',
 'continue_generate_map',
 'delete_dnd_mode',
 'gesture_name',
 'info_action_status',
 'info_clean_completed_space',
 'info_clean_next_space',
 'info_clean_progress',
 'info_clean_space',
 'info_clean_unfinished_space',
 'info_complete_time',
 'info_consumable',
 'info_elec_status',
 'info_error_status',
 'info_location_status',
 'info_todo_time',
 'move',
 'move_location',
 'open_ai_recognition',
 'open_carpet_avoidance',
 'open_carpet_pressurization',
 'open_childlock',
 'open_dnd_mode',
 'open_laser_assistant',
 'open_station_light',
 'pause_back_station',
 'pause_clean_roller',
 'pause_collect_dust',
 'pause_drying',
 'pause_exit_duck',
 'pause_generate_map',
 'set_dn

In [14]:
dataDf[dataDf["intent"].isin(intentList)].drop_duplicates(["query"], keep="first").to_csv("intent_static_narwal_6-11.tsv", index=False, sep="\t")

In [16]:
intentDataNum = []
for intent in intentList:
    dataNum = len(dataDf[dataDf["intent"] == intent])
    intentDataNum.append([intent, dataNum])
intentDataDf = pd.DataFrame(intentDataNum, columns=["intent", "count"])
intentDataDf.sort_values("count", ascending=False).to_csv("intent_narwal_data_num_6-11.tsv", index=False, sep="\t")


### 将石头与云鲸合并再检测

In [13]:
narwalDf = pd.read_csv("./results/intent_static_narwal_6-11.tsv", sep="\t")
roborock = pd.read_csv("./results/intent_roborock_data.tsv", sep="\t")
mergeData = pd.concat([narwalDf, roborock])
mergeData.head()

,query,domain,intent,slots
0,嗯帮我返回基站,narwal_other_function,start_back_station,[]
1,回去充电,narwal_other_function,start_back_station,[]
2,来我这里扫一下,narwal_clean_function,come_here_sweep,[]
3,来我这里打扫,narwal_clean_function,come_here_clean,[]
4,返回基站,narwal_other_function,start_back_station,[]


In [14]:
import yaml
import os
def parser_yml(ymlDir):
    ymlFilePaths = [ymlDir + "/" + ymlName for ymlName in os.listdir(ymlDir)]
    intentData = {}
    for ymlPath in ymlFilePaths:
        with open(ymlPath, 'r') as file:
            data = yaml.safe_load(file)
            topKey = list(data.keys())[0]
            intentData.setdefault(topKey, list(data.get(topKey).keys()))
    return intentData
dataInput = "/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/dags/temp/task43 query_slots_sta/inputData"
intentDict = parser_yml(dataInput)
intentList = []
for intentValues in intentDict.values(): 
    intentList += intentValues
intentList

['close_ai_recognition',
 'close_carpet_avoidance',
 'close_carpet_pressurization',
 'close_childlock',
 'close_dnd_mode',
 'close_laser_assistant',
 'close_station_light',
 'close_voice_assistant',
 'continue_back_station',
 'continue_clean_roller',
 'continue_collect_dust',
 'continue_drying',
 'continue_exit_duck',
 'continue_generate_map',
 'delete_dnd_mode',
 'gesture_name',
 'info_action_status',
 'info_clean_completed_space',
 'info_clean_next_space',
 'info_clean_progress',
 'info_clean_space',
 'info_clean_unfinished_space',
 'info_complete_time',
 'info_consumable',
 'info_elec_status',
 'info_error_status',
 'info_location_status',
 'info_todo_time',
 'move',
 'move_location',
 'open_ai_recognition',
 'open_carpet_avoidance',
 'open_carpet_pressurization',
 'open_childlock',
 'open_dnd_mode',
 'open_laser_assistant',
 'open_station_light',
 'pause_back_station',
 'pause_clean_roller',
 'pause_collect_dust',
 'pause_drying',
 'pause_exit_duck',
 'pause_generate_map',
 'set_dn

In [15]:
mergeData[mergeData["intent"].isin(intentList)].drop_duplicates(["query"], keep="first").to_csv("intent_static_data.tsv", index=False, sep="\t")

In [16]:
intentDataNum = []
for intent in intentList:
    dataNum = len(mergeData[mergeData["intent"] == intent])
    intentDataNum.append([intent, dataNum])
intentDataDf = pd.DataFrame(intentDataNum, columns=["intent", "count"])
intentDataDf.sort_values("count", ascending=False).to_csv("intent_data_.tsv", index=False, sep="\t")